In [10]:
import os
import sys
import importlib
from pyexpat import features

import numpy as np
from matplotlib import get_data_path

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))
from utilities.path_utils import *
from utilities.load_utils import *
import pandas as pd

In [2]:
pc1 = pd.read_csv('../src/poland_postal_codes.csv')
pc2 = pd.read_csv('../src/poland_postal_codes_1.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../src/poland_postal_codes.csv'

In [27]:
data, _, _, features = load_data('netrisk_casco_v12')

In [10]:
pc = pd.concat([pc1, pc2])
pc = pc[pc['voivodeship'] != '—']
pc = pc[pc['postal_code_population'] != '—']
pc = pc[pc['postal_code_area'] != '—']
pc = pc[pc['city'] != '—']
pc['postal_code_area'] = pc['postal_code_area'].map(lambda x : float(x.split(' ')[0].replace(',', '.')))
pc['postal_code_population'] = pc['postal_code_population'].map(lambda x : int(float(x)))
pc

,postal_code,city,voivodeship,postal_code_population,postal_code_area
1,00-001,Warsaw,Masovian Voivodeship,4562,17.451
2,00-002,Warsaw,Masovian Voivodeship,1873,6.099
3,00-003,Warsaw,Masovian Voivodeship,1873,5.046
4,00-004,Warsaw,Masovian Voivodeship,930,3.197
5,00-005,Warsaw,Masovian Voivodeship,474,3.269
...,...,...,...,...,...
4034,99-412,Czerniew,Łódź Voivodeship,3483,75.900
4038,99-417,"Humin, Poland",Łódź Voivodeship,3730,101.100
4040,99-420,Gmina Łyszkowice,Łódź Voivodeship,7576,129.100
4041,99-423,Stary Waliszew,Łódź Voivodeship,5596,166.400


In [13]:
postal_code_data = pd.read_csv('../data/external/distributions/punkta/other/poland_postal_codes.csv').iloc[:, 2:]

In [18]:
pd.merge(postal_code_data, pc[['postal_code', 'postal_code_population', 'postal_code_area']], on = 'postal_code', how = 'left').to_csv('../data/external/distributions/punkta/other/poland_postal_codes.csv', index = False)

In [4]:
data = pd.read_parquet(get_raw_data_path('punkta_data_processed_2024_07_01-2024_11_01', '.parquet'))

In [14]:
data['time_delta'] = (data['policy_start_date'] - pd.to_datetime(data['calculation_time'])).dt.days
data[data['time_delta'] < 100]['MTU24-(mtpl)-price'].mean()

1494.7131608548932

In [20]:
data_name = 'punkta_v6'
target_variables = ['TRASTI-(mtpl)(green_card)-price', 'FITCASCO-(mtpl)-price', 'BESAFE-(mtpl)-price', 'YOU_CAN_DRIVE-(mtpl)-price', 'MTU24-(mtpl)-price']


In [24]:
target = 'log_BESAFE-(mtpl)-price'
outlier = f'is_outlier_per_{target.strip('log_')}'
data, features_info, features_on_top, features_model = load_data(data_name)

In [25]:
data

,"UNIQA-(mtpl (6-10 year old car))(casco_repair_shop,500)(assistance_medium)(personal_accident_all_passengers)-price",is_outlier_per_TRASTI-(mtpl)(casco_authorized_repair_shop)(green_card)(personal_accident_all_passengers50000)(assistance500)(sam zastępczy 7 day)-price,PROAMA-(mtpl)(personal_accident_all_passengers)(assistance_medium)-price,BESAFE-(mtpl)(personal_accident_all_passengers)(assistance_standard)(casco_authorized_repair_shop)-price,log_EUROINS-(mtpl)(personal_accident_all_passengers)(assistance)-price,log_PROAMA-(personal_accident_all_passengers)-isolated_price,log_BESAFE-(mtpl)(personal_accident_all_passengers)(assistance_premium)(casco_repair_shop part.)-price,"log_UNIQA-(casco_authorized_repair_shop,500)-isolated_price",is_outlier_per_YOU_CAN_DRIVE-(window_insurance)-isolated_price,is_outlier_per_MTU24-(mtpl)(personal_accident_all_passengers1)-price,...,log_MTU24-(mtpl)(assistance1)(personal_accident_all_passengers1)(minicasco(żywioły)(theft_protection))-price,BESAFE-(casco_repair_shop part.(amortyzcascoja))-isolated_price,"UNIQA-(mtpl (0-5 year old car))(casco_repair_shop,500)(assistance_medium)(personal_accident_all_passengers)-price",log_BESAFE-(mtpl)(assistance_premium)(personal_accident_all_passengers)-price,is_outlier_per_TRASTI-(mtpl)(green_card)-price,"player_1-(mtpl)(green_card)(assistance basic)(personal_accident_all_passengers)(casco ""ekonomiczne"" (udział własny 10% zamienniki,repair_shop,bez amortyzcascoji))-price","is_outlier_per_MTU24-(casco ""ekonomiczne"" (sieć partnerska))-isolated_price",is_outlier_per_EUROINS-(mtpl)(personal_accident_all_passengers(5.000))(assistance awaria)(tyres)(akumulator)-price,is_outlier_per_UNIQA-(mtpl)(assistance_medium)(assistance_prestige personal_accident_all_passengers10000)-price,is_outlier_per_LINK4-(mtpl)(assistance)-price
0,NaN,False,NaN,NaN,6.949598,NaN,NaN,NaN,False,False,...,NaN,NaN,NaN,NaN,False,2043.0,False,False,False,False
2,NaN,False,NaN,NaN,7.116654,NaN,NaN,NaN,False,False,...,7.364547,756.41,NaN,NaN,False,2109.0,False,False,False,False
3,1627.0,False,1729.0,3411.65,7.156535,-inf,8.151293,NaN,False,False,...,NaN,NaN,NaN,7.215225,False,3747.0,False,False,False,False
4,NaN,False,1038.0,2266.91,7.046142,-inf,7.807103,NaN,False,False,...,6.591674,NaN,NaN,6.963190,False,2768.0,False,False,False,False
5,NaN,False,1713.0,4011.94,NaN,-inf,8.290880,NaN,False,False,...,8.071843,NaN,NaN,7.142060,False,4305.0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66245,2434.0,False,NaN,NaN,7.402927,NaN,NaN,NaN,False,True,...,8.133000,NaN,NaN,NaN,False,NaN,False,True,False,True
66246,NaN,False,NaN,NaN,7.009960,NaN,NaN,NaN,False,True,...,NaN,1593.11,NaN,NaN,False,NaN,False,False,False,True
66247,NaN,False,NaN,NaN,8.965699,NaN,NaN,NaN,False,False,...,7.991254,4850.00,NaN,NaN,False,11261.0,False,True,False,False
66249,NaN,False,NaN,NaN,7.461290,NaN,NaN,NaN,False,True,...,NaN,3435.74,NaN,NaN,True,NaN,False,False,False,True


In [44]:
data['vehicle_power_cut'] = pd.cut(data['vehicle_power'].values, bins=20)
data['contractor_age_cut'] = pd.cut(data['contractor_age'].values, bins=20)

threshold = 0.9
data['vehicle_power_threshold'] = data.groupby('vehicle_power_cut', observed=False)['BESAFE-(mtpl)-price'].transform(lambda x: x.quantile(threshold))
data['contractor_age_threshold'] = data.groupby('contractor_age_cut', observed=False)['BESAFE-(mtpl)-price'].transform(lambda x: x.quantile(threshold))


In [47]:
data.groupby('vehicle_power_cut', observed=False)['BESAFE-(mtpl)-price'].transform(lambda x: x.quantile(threshold))

0        2397.716
2        2547.851
3        2547.851
4        2547.851
5        2963.594
           ...   
66245    2397.716
66246    2556.392
66247    2963.594
66249    2963.594
66252    2397.716
Name: BESAFE-(mtpl)-price, Length: 55497, dtype: float64

In [37]:
data.groupby('vehicle_power_cut', observed=False).agg({'BESAFE-(mtpl)-price' : lambda x : x.quantile(threshold)})

,BESAFE-(mtpl)-price
vehicle_power_cut,
"(19.542, 42.9]",1898.034
"(42.9, 65.8]",2397.716
"(65.8, 88.7]",2556.392
"(88.7, 111.6]",2547.851
"(111.6, 134.5]",2963.594
"(134.5, 157.4]",2711.820
"(157.4, 180.3]",3310.793
"(180.3, 203.2]",3401.434
"(203.2, 226.1]",5186.740


In [46]:
data[['vehicle_power_cut', 'contractor_age_cut', 'vehicle_power_threshold', 'contractor_age_threshold', 'BESAFE-(mtpl)-price', 'is_outlier_per_BESAFE-(mtpl)-price' ]]

,vehicle_power_cut,contractor_age_cut,vehicle_power_threshold,contractor_age_threshold,BESAFE-(mtpl)-price,is_outlier_per_BESAFE-(mtpl)-price
0,"(42.9, 65.8]","(33.6, 37.5]",2397.716,2077.966,NaN,False
2,"(88.7, 111.6]","(37.5, 41.4]",2547.851,2148.645,949.61,False
3,"(88.7, 111.6]","(53.1, 57.0]",2547.851,2357.460,887.98,False
4,"(88.7, 111.6]","(41.4, 45.3]",2547.851,2309.197,585.00,False
5,"(111.6, 134.5]","(49.2, 53.1]",2963.594,2330.774,792.03,False
...,...,...,...,...,...,...
66245,"(42.9, 65.8]","(41.4, 45.3]",2397.716,2309.197,NaN,False
66246,"(65.8, 88.7]","(49.2, 53.1]",2556.392,2330.774,1114.49,False
66247,"(111.6, 134.5]","(33.6, 37.5]",2963.594,2077.966,8000.00,True
66249,"(111.6, 134.5]","(33.6, 37.5]",2963.594,2077.966,1899.64,True


In [38]:
data.groupby('contractor_age_cut', observed=False).agg({'BESAFE-(mtpl)-price' : lambda x : x.mean()})

,BESAFE-(mtpl)-price
contractor_age_cut,
"(17.922, 21.9]",5119.112066
"(21.9, 25.8]",3071.143474
"(25.8, 29.7]",1783.569740
"(29.7, 33.6]",1354.503880
"(33.6, 37.5]",1227.703750
"(37.5, 41.4]",1214.413564
"(41.4, 45.3]",1258.003715
"(45.3, 49.2]",1295.980025
"(49.2, 53.1]",1249.897985


In [39]:
data['BESAFE-(mtpl)-price'].skew()

3.236805979232619

In [41]:
np.log(data['log_BESAFE-(mtpl)-price']).skew()

1.0261056248953861

In [42]:
data['log_BESAFE-(mtpl)-price'].skew()

1.258990147769411

In [26]:
data[data[outlier]][target_variables]

,TRASTI-(mtpl)(green_card)-price,FITCASCO-(mtpl)-price,BESAFE-(mtpl)-price,YOU_CAN_DRIVE-(mtpl)-price,MTU24-(mtpl)-price
40,1637.0,NaN,1744.48,NaN,2181.0
55,2175.0,1289.56,2019.39,NaN,2045.0
113,1292.0,NaN,2278.70,1312.0,1312.0
224,802.0,689.31,3967.64,NaN,1885.0
225,2565.0,889.92,1779.85,NaN,2435.0
...,...,...,...,...,...
66232,2009.0,1891.29,3705.49,NaN,3852.0
66241,6390.0,NaN,4923.35,5919.0,5978.0
66242,609.0,NaN,2291.72,NaN,2592.0
66247,NaN,7777.00,8000.00,NaN,1955.0


In [27]:
data[(np.exp(data[target]) == 8000) & (data[outlier])]

,"UNIQA-(mtpl (6-10 year old car))(casco_repair_shop,500)(assistance_medium)(personal_accident_all_passengers)-price",is_outlier_per_TRASTI-(mtpl)(casco_authorized_repair_shop)(green_card)(personal_accident_all_passengers50000)(assistance500)(sam zastępczy 7 day)-price,PROAMA-(mtpl)(personal_accident_all_passengers)(assistance_medium)-price,BESAFE-(mtpl)(personal_accident_all_passengers)(assistance_standard)(casco_authorized_repair_shop)-price,log_EUROINS-(mtpl)(personal_accident_all_passengers)(assistance)-price,log_PROAMA-(personal_accident_all_passengers)-isolated_price,log_BESAFE-(mtpl)(personal_accident_all_passengers)(assistance_premium)(casco_repair_shop part.)-price,"log_UNIQA-(casco_authorized_repair_shop,500)-isolated_price",is_outlier_per_YOU_CAN_DRIVE-(window_insurance)-isolated_price,is_outlier_per_MTU24-(mtpl)(personal_accident_all_passengers1)-price,...,log_MTU24-(mtpl)(assistance1)(personal_accident_all_passengers1)(minicasco(żywioły)(theft_protection))-price,BESAFE-(casco_repair_shop part.(amortyzcascoja))-isolated_price,"UNIQA-(mtpl (0-5 year old car))(casco_repair_shop,500)(assistance_medium)(personal_accident_all_passengers)-price",log_BESAFE-(mtpl)(assistance_premium)(personal_accident_all_passengers)-price,is_outlier_per_TRASTI-(mtpl)(green_card)-price,"player_1-(mtpl)(green_card)(assistance basic)(personal_accident_all_passengers)(casco ""ekonomiczne"" (udział własny 10% zamienniki,repair_shop,bez amortyzcascoji))-price","is_outlier_per_MTU24-(casco ""ekonomiczne"" (sieć partnerska))-isolated_price",is_outlier_per_EUROINS-(mtpl)(personal_accident_all_passengers(5.000))(assistance awaria)(tyres)(akumulator)-price,is_outlier_per_UNIQA-(mtpl)(assistance_medium)(assistance_prestige personal_accident_all_passengers10000)-price,is_outlier_per_LINK4-(mtpl)(assistance)-price
521,7504.0,False,NaN,NaN,7.458815,NaN,NaN,NaN,False,False,...,8.152198,10700.00,NaN,NaN,False,NaN,False,True,False,False
528,NaN,True,8138.0,13035.92,8.965699,2.995732,9.457242,7.627544,False,True,...,7.841100,NaN,2824.0,9.044522,True,5441.0,False,True,False,True
865,NaN,True,NaN,NaN,8.965699,NaN,NaN,NaN,False,True,...,9.087042,4188.15,NaN,NaN,True,3913.0,False,True,False,True
1469,3075.0,True,NaN,NaN,8.965699,NaN,NaN,NaN,False,False,...,NaN,1650.00,NaN,NaN,True,NaN,False,False,False,False
1816,NaN,True,NaN,NaN,8.965699,NaN,NaN,NaN,False,True,...,NaN,1800.00,NaN,NaN,True,NaN,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66205,NaN,False,NaN,NaN,8.967103,NaN,NaN,NaN,False,True,...,NaN,3900.00,NaN,NaN,False,NaN,False,False,False,False
66209,NaN,False,2499.0,24839.00,8.965699,2.995732,10.133488,NaN,False,False,...,8.149313,NaN,NaN,9.044522,False,9890.0,True,True,False,False
66221,4570.0,False,NaN,NaN,8.967103,NaN,NaN,NaN,False,True,...,8.674539,5024.60,NaN,NaN,False,NaN,False,False,False,False
66226,NaN,True,33862.0,23164.00,8.965699,2.995732,10.053196,NaN,False,True,...,9.564512,NaN,NaN,9.044522,True,15480.0,True,True,False,True


In [53]:
def make_is_outlier_columns(data, target_variable):
    data['vehicle_power_cut'] = pd.cut(data['vehicle_power'].values, bins=20)
    data['contractor_age_cut'] = pd.cut(data['contractor_age'].values, bins=20)

    vehicle_power_threshold = data.groupby('vehicle_power_cut', observed=False)[target_variable].transform(lambda x: x.quantile(0.85))
    contractor_age_threshold = data.groupby('contractor_age_cut', observed=False)[target_variable].transform(lambda x: x.quantile(0.85))

    outlier_column_name = f'is_outlier_per_{target_variable}'
    data[outlier_column_name] = (
        (data[target_variable] >= vehicle_power_threshold) &
        (data[target_variable] >= contractor_age_threshold)
    )

    return data, outlier_column_name


In [54]:
data, col = make_is_outlier_columns(data, 'BESAFE-(mtpl)-price')

In [55]:
data[col].value_counts()

is_outlier_per_BESAFE-(mtpl)-price
False    49245
True      6252
Name: count, dtype: int64

In [56]:
data[(data['BESAFE-(mtpl)-price'] == 8000) & (data[col])][target_variables]

,TRASTI-(mtpl)(green_card)-price,FITCASCO-(mtpl)-price,BESAFE-(mtpl)-price,YOU_CAN_DRIVE-(mtpl)-price,MTU24-(mtpl)-price
138,2453.0,NaN,8000.0,4278.0,4321.0
188,2074.0,NaN,8000.0,NaN,9595.0
191,13325.0,7777.0,8000.0,9220.0,9312.0
192,13325.0,7777.0,8000.0,9220.0,9312.0
402,4646.0,NaN,8000.0,4160.0,4202.0
...,...,...,...,...,...
66205,2888.0,NaN,8000.0,10349.0,10453.0
66209,NaN,NaN,8000.0,NaN,1267.0
66221,2060.0,NaN,8000.0,5392.0,5446.0
66226,10123.0,7777.0,8000.0,NaN,11135.0


In [57]:
data[(data['BESAFE-(mtpl)-price'] == 8000)][target_variables]

,TRASTI-(mtpl)(green_card)-price,FITCASCO-(mtpl)-price,BESAFE-(mtpl)-price,YOU_CAN_DRIVE-(mtpl)-price,MTU24-(mtpl)-price
138,2453.0,NaN,8000.0,4278.0,4321.0
188,2074.0,NaN,8000.0,NaN,9595.0
191,13325.0,7777.0,8000.0,9220.0,9312.0
192,13325.0,7777.0,8000.0,9220.0,9312.0
402,4646.0,NaN,8000.0,4160.0,4202.0
...,...,...,...,...,...
66205,2888.0,NaN,8000.0,10349.0,10453.0
66209,NaN,NaN,8000.0,NaN,1267.0
66221,2060.0,NaN,8000.0,5392.0,5446.0
66226,10123.0,7777.0,8000.0,NaN,11135.0


In [60]:
data[[col for col in data.columns if col.endswith('-price') and not col.startswith('is_outlier_per_')]].skew()

UNIQA-(mtpl (6-10 year old car))(casco_repair_shop,500)(assistance_medium)(personal_accident_all_passengers)-price                                                          2.792738
PROAMA-(mtpl)(personal_accident_all_passengers)(assistance_medium)-price                                                                                                    6.045090
BESAFE-(mtpl)(personal_accident_all_passengers)(assistance_standard)(casco_authorized_repair_shop)-price                                                                    3.742020
log_EUROINS-(mtpl)(personal_accident_all_passengers)(assistance)-price                                                                                                      1.883237
log_BESAFE-(mtpl)(personal_accident_all_passengers)(assistance_premium)(casco_repair_shop part.)-price                                                                      1.293729
                                                                                               

In [65]:
data[target_variables].skew()

TRASTI-(mtpl)(green_card)-price    5.313506
FITCASCO-(mtpl)-price              8.056794
BESAFE-(mtpl)-price                3.236806
YOU_CAN_DRIVE-(mtpl)-price         1.960514
MTU24-(mtpl)-price                 3.124248
dtype: float64

In [64]:
data['WIENER-(mtpl)(personal_accident_all_passengers)(assistance_standardpl)-price'].describe()

count     52782.000000
mean       1834.125990
std        2453.704233
min         439.000000
25%         988.000000
50%        1347.000000
75%        1948.750000
max      188029.000000
Name: WIENER-(mtpl)(personal_accident_all_passengers)(assistance_standardpl)-price, dtype: float64

In [18]:
features_model

['is_outlier_per_WIENER-(mtpl)(personal_accident_all_passengers)(assistance)-price',
 'is_outlier_per_UNIQA-(mtpl)(green_card)(assistance medium)(assistance_premium_eu)(casco authorized_repair_shop_pl (brak u.wł, 0%))(personal_accident_all_passengers30)-price',
 'is_outlier_per_EUROINS-(mtpl)(personal_accident_all_passengers)(assistance)(tyres)(akumulator)-price',
 'is_outlier_per_FITCASCO-(mtpl)(personal_accident_all_passengers)(cascomini)-price',
 'vehicle_parking_place',
 'is_outlier_per_PROAMA-(minicasco(repair_shop))-isolated_price',
 'is_outlier_per_LINK4-(mtpl)(personal_accident_all_passengers 50.000)(assistance auto assistanceistance komfort (w, a, k, opona))(casco (authorized_repair_shop, stała su, oryginalne, u.wł 500 zł, bez amortyzcascoji))-price',
 'is_outlier_per_FITCASCO-(mtpl)(personal_accident_all_passengers)(cascomini_1)-price',
 'is_outlier_per_TRASTI-(mtpl)(green_card)(personal_accident_all_passengers100000)(assistance1500)(sam zastępczy 21 day)-price',
 'is_outlier

In [21]:
data[data['is_outlier_per_BESAFE-(mtpl)-price']][target_variables + ['number_of_damages_caused_in_last_5_years']]

,TRASTI-(mtpl)(green_card)-price,FITCASCO-(mtpl)-price,BESAFE-(mtpl)-price,YOU_CAN_DRIVE-(mtpl)-price,MTU24-(mtpl)-price,number_of_damages_caused_in_last_5_years
224,802.0,689.31,3967.64,NaN,1885.0,0
305,1241.0,797.29,2892.73,NaN,1742.0,1
504,2507.0,1347.49,3978.19,NaN,1647.0,0
519,NaN,NaN,6115.16,NaN,5691.0,0
520,NaN,NaN,5997.43,NaN,5691.0,0
...,...,...,...,...,...,...
66209,NaN,NaN,8000.00,NaN,1267.0,0
66221,2060.0,NaN,8000.00,5392.0,5446.0,0
66226,10123.0,7777.00,8000.00,NaN,11135.0,3
66232,2009.0,1891.29,3705.49,NaN,3852.0,0


In [27]:
data[(data['BESAFE-(mtpl)-price'] == 8000) & (data['is_outlier_per_BESAFE-(mtpl)-price'])][target_variables + ['number_of_damages_caused_in_last_5_years', 'vehicle_power', 'contractor_age']]

,TRASTI-(mtpl)(green_card)-price,FITCASCO-(mtpl)-price,BESAFE-(mtpl)-price,YOU_CAN_DRIVE-(mtpl)-price,MTU24-(mtpl)-price,number_of_damages_caused_in_last_5_years,vehicle_power,contractor_age
521,NaN,1810.16,8000.0,NaN,2085.0,0,220.0,35.0
528,5924.0,NaN,8000.0,NaN,2317.0,0,60.0,50.0
865,4671.0,NaN,8000.0,NaN,7941.0,4,77.0,44.0
1469,1984.0,NaN,8000.0,NaN,NaN,1,48.0,72.0
1816,8260.0,NaN,8000.0,7577.0,7652.0,0,74.0,29.0
...,...,...,...,...,...,...,...,...
66189,1603.0,946.74,8000.0,NaN,8932.0,2,55.0,48.0
66209,NaN,NaN,8000.0,NaN,1267.0,0,120.0,53.0
66221,2060.0,NaN,8000.0,5392.0,5446.0,0,73.0,25.0
66226,10123.0,7777.00,8000.0,NaN,11135.0,3,72.0,37.0


In [26]:
data[data['number_of_damages_caused_in_last_5_years'] == 5][['number_of_damages_caused_in_last_5_years'] + target_variables].dropna(axis = 0, how = 'all')

,number_of_damages_caused_in_last_5_years,TRASTI-(mtpl)(green_card)-price,FITCASCO-(mtpl)-price,BESAFE-(mtpl)-price,YOU_CAN_DRIVE-(mtpl)-price
1334,5,816.0,NaN,NaN,NaN
1716,5,13580.0,NaN,8000.00,NaN
2024,5,531.0,NaN,1567.38,NaN
2943,5,NaN,7777.00,8000.00,NaN
3337,5,830.0,NaN,1509.05,NaN
...,...,...,...,...,...
56841,5,1241.0,796.47,990.27,NaN
58234,5,NaN,1180.14,NaN,NaN
58264,5,2073.0,1256.35,6183.39,NaN
59229,5,5945.0,NaN,4447.91,NaN


In [28]:
data.groupby('number_of_damages_caused_in_last_5_years')[target_variables[1]].describe()

,count,mean,std,min,25%,50%,75%,max
number_of_damages_caused_in_last_5_years,,,,,,,,
0,13501.0,840.885695,586.310437,411.00,585.5700,724.700,929.6900,7788.00
1,2498.0,921.794131,642.112226,411.00,650.1500,813.910,1008.0075,7788.00
2,782.0,1057.993913,931.309958,411.00,683.7325,871.725,1099.9225,7777.00
3,147.0,1167.188027,1209.226398,422.67,691.9500,877.840,1097.4500,7777.00
4,70.0,946.661286,347.489918,457.27,659.2950,906.300,1166.5175,1995.02
5,24.0,1443.678750,1971.194259,528.19,665.2300,879.335,1021.1325,7777.00
6,18.0,1342.976111,1671.001127,504.81,636.2625,921.405,1202.5475,7777.00
7,6.0,1311.833333,818.173560,840.37,917.4550,1024.680,1104.4100,2968.57
8,3.0,3275.270000,3898.860883,980.40,1024.4050,1068.410,4422.7050,7777.00


In [30]:
data[data['number_of_damages_caused_in_last_5_years'] == 9][target_variables]

,TRASTI-(mtpl)(green_card)-price,FITCASCO-(mtpl)-price,BESAFE-(mtpl)-price,YOU_CAN_DRIVE-(mtpl)-price
7703,NaN,NaN,8000.00,NaN
44869,9034.0,7777.00,3295.71,NaN
47705,525.0,NaN,552.87,NaN
51159,NaN,1691.91,8000.00,10219.0


In [33]:
data[target_variables[2]].value_counts()

BESAFE-(mtpl)-price
8000.00    973
400.00     701
404.00       7
660.33       7
627.90       6
          ... 
1288.62      1
819.97       1
996.46       1
2593.13      1
1314.65      1
Name: count, Length: 41901, dtype: int64

In [35]:
data[data[target_variables[2]] > 2000]

,UNIQA-(mtpl (6-10 year old car))(casco_authorized_repair_shop)(assistance_medium)(personal_accident_all_passengers)-price,WIENER-(mtpl)(assistance)-price,BESAFE-(mtpl)(assistance_basic)(casco_repair_shop part.(amortyzcascoja))-price,BESAFE-(mtpl)(personal_accident_all_passengers)(assistance_standard)(casco_repair_shop part.)-price,GENERALI-(mtpl)(green_card)(personal_accident_all_passengers)(assistancekomfort (nowe mtpl))-price,FITCASCO-(mtpl)(personal_accident_all_passengers)(cascomini)-price,"WIENER-(casco (cash_settlement, bez theft_protectiony, uw500))-isolated_price",LINK4-(mtpl)(personal_accident_all_passengers)(assistance)-price,TRASTI-(mtpl)(green_card)(personal_accident_all_passengers50000)(assistance500)(sam zastępczy 14 day)-price,"WIENER-(mtpl)(casco (cash_settlement, uwł.1000))(personal_accident_all_passengers)(auto assistanceistance plus standard polska)-price",...,player_1-(mtpl)(green_card)-price,"LINK4-(mtpl)(personal_accident_all_passengers 50.000)(assistance auto assistanceistance komfort (w, a, k, opona))(casco (authorized_repair_shop, stała su, oryginalne, u.wł 500 zł, bez amortyzcascoji))-price",UNIQA-(assistance_prestige personal_accident_all_passengers10000)-isolated_price,EUROINS-(personal_accident_all_passengers)-isolated_price,MTU24-(assistance1)-isolated_price,LINK4-(mtpl)(personal_accident_all_passengers)(assistance)(window_insurance)-price,"UNIQA-(casco_authorized_repair_shop,500)-isolated_price",MTU24-(mtpl)(assistance2)(personal_accident_all_passengers1)-price,MTU24-(mtpl)(cascozywioły)(cascokr)-price,EUROINS-(casco cash_settlement)-isolated_price
22,2996.0,2489.0,NaN,4532.29,NaN,NaN,146.0,2848.93,2816.0,2795.0,...,1919.0,NaN,NaN,NaN,49.0,2938.93,NaN,2215.0,2627.0,NaN
25,NaN,12716.0,NaN,10685.85,6857.0,7883.0,-138.0,3903.98,9889.0,12971.0,...,3089.0,7451.39,NaN,38.0,62.0,NaN,NaN,7517.0,8528.0,NaN
26,3184.0,2628.0,NaN,4576.03,NaN,NaN,179.0,2799.69,2889.0,2969.0,...,1919.0,NaN,NaN,NaN,50.0,NaN,NaN,2295.0,2773.0,NaN
27,NaN,1888.0,4393.23,NaN,1870.0,NaN,250.0,1434.83,4958.0,2277.0,...,1543.0,NaN,NaN,38.0,112.0,NaN,NaN,3417.0,3399.0,1174.04
44,NaN,3348.0,8587.22,NaN,NaN,NaN,543.0,NaN,2038.0,4021.0,...,NaN,NaN,NaN,NaN,118.0,NaN,NaN,3984.0,4150.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59540,NaN,1846.0,12377.00,NaN,NaN,NaN,-49.0,1927.42,3106.0,1949.0,...,6404.0,NaN,NaN,NaN,203.0,NaN,NaN,1832.0,NaN,NaN
59542,NaN,2987.0,5015.67,NaN,NaN,NaN,389.0,2005.65,2556.0,3510.0,...,NaN,NaN,NaN,38.0,232.0,NaN,NaN,4503.0,NaN,1083.85
59545,NaN,3360.0,5609.54,NaN,4203.0,NaN,NaN,2717.93,NaN,NaN,...,NaN,NaN,NaN,38.0,253.0,NaN,NaN,8969.0,NaN,NaN
59547,NaN,NaN,NaN,NaN,10533.0,NaN,NaN,NaN,NaN,NaN,...,7576.0,NaN,NaN,38.0,320.0,NaN,NaN,11051.0,NaN,NaN


In [34]:
data[features_model].drop_duplicates()

,vehicle_engine_size,vehicle_maker,vehicle_weight_max,vehicle_weight_min,licence_at_age,vehicle_make_year,latitude,driver_experience,owner_driver_same,county,mileage_domestic,number_of_damages_caused_in_last_5_years,longitude,voivodeship,contractor_age,vehicle_fuel_type,vehicle_power,vehicle_parking_place
0,1390,SEAT,1543,1028,18.0,2009,54.4257,16.0,yes,Gdańsk,2500.0,0,18.4685,Pomerania,34.0,petrol,63.0,common_garage
1,1998,MAZDA,2020,1360,22.0,2018,54.4257,18.0,yes,Gdańsk,2500.0,0,18.4685,Pomerania,40.0,petrol,121.0,individual_garage
2,1870,RENAULT,1968,1328,24.0,2010,54.4257,16.0,yes,Gdańsk,8750.0,1,18.4685,Pomerania,40.0,diesel,96.0,individual_garage
3,1595,MERCEDES-BENZ,1935,1370,17.0,2017,54.4257,39.0,yes,Gdańsk,2500.0,0,18.4685,Pomerania,56.0,petrol,90.0,individual_garage
4,1498,VOLKSWAGEN,2050,1454,17.0,2022,54.4257,27.0,yes,Gdańsk,8750.0,0,18.4685,Pomerania,44.0,petrol,110.0,individual_garage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59545,1796,OPEL,1655,1135,27.0,2003,52.3898,0.0,yes,Poznań,6250.0,0,16.8858,Greater Poland,27.0,petrol,92.0,street
59546,996,SUZUKI,1485,975,24.0,2009,52.2447,15.0,yes,Warszawa,2500.0,0,20.9448,Mazovia,39.0,petrol,48.0,unguarded_parking
59548,1598,RENAULT,1877,1320,17.0,2011,51.2651,27.0,yes,Lublin,8750.0,0,22.5083,Lublin,44.0,petrol,81.0,guarded_parking
59549,1991,MERCEDES-BENZ,2050,1555,34.0,2018,51.1095,18.0,yes,Wrocław,8750.0,0,17.0342,Lower Silesia,52.0,petrol,135.0,street


In [35]:
data[features_model]

,vehicle_engine_size,vehicle_maker,vehicle_weight_max,vehicle_weight_min,licence_at_age,vehicle_make_year,latitude,driver_experience,owner_driver_same,county,mileage_domestic,number_of_damages_caused_in_last_5_years,longitude,voivodeship,contractor_age,vehicle_fuel_type,vehicle_power,vehicle_parking_place
0,1390,SEAT,1543,1028,18.0,2009,54.4257,16.0,yes,Gdańsk,2500.0,0,18.4685,Pomerania,34.0,petrol,63.0,common_garage
1,1998,MAZDA,2020,1360,22.0,2018,54.4257,18.0,yes,Gdańsk,2500.0,0,18.4685,Pomerania,40.0,petrol,121.0,individual_garage
2,1870,RENAULT,1968,1328,24.0,2010,54.4257,16.0,yes,Gdańsk,8750.0,1,18.4685,Pomerania,40.0,diesel,96.0,individual_garage
3,1595,MERCEDES-BENZ,1935,1370,17.0,2017,54.4257,39.0,yes,Gdańsk,2500.0,0,18.4685,Pomerania,56.0,petrol,90.0,individual_garage
4,1498,VOLKSWAGEN,2050,1454,17.0,2022,54.4257,27.0,yes,Gdańsk,8750.0,0,18.4685,Pomerania,44.0,petrol,110.0,individual_garage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59545,1796,OPEL,1655,1135,27.0,2003,52.3898,0.0,yes,Poznań,6250.0,0,16.8858,Greater Poland,27.0,petrol,92.0,street
59546,996,SUZUKI,1485,975,24.0,2009,52.2447,15.0,yes,Warszawa,2500.0,0,20.9448,Mazovia,39.0,petrol,48.0,unguarded_parking
59548,1598,RENAULT,1877,1320,17.0,2011,51.2651,27.0,yes,Lublin,8750.0,0,22.5083,Lublin,44.0,petrol,81.0,guarded_parking
59549,1991,MERCEDES-BENZ,2050,1555,34.0,2018,51.1095,18.0,yes,Wrocław,8750.0,0,17.0342,Lower Silesia,52.0,petrol,135.0,street
